In [1]:
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)

In [4]:
shape = ((flights_df.count(), len(flights_df.columns)))
print('The shape of flights_df:', shape)

The shape of flights_df: (5819079, 31)

In [5]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in flights_df.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
 0| 0| 0| 0| 0| 0| 14721| 0| 0| 0| 86153| 86153| 89047| 89047| 6| 105071| 105071| 0| 92513| 92513| 0| 92513| 105071| 0| 0| 5729195| 4755640| 4755640| 4755640| 4755640| 4755640|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+

In [6]:
flights_df = flights_df.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')

flights_df = flights_df.filter((fn.col('CANCELLED')==0))

flights_df = flights_df.withColumn("Flight_Delayed", fn.when(fn.col("DEPARTURE_DELAY")<15, 0).otherwise(1))

from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)

from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

flights_df = flights_df.drop('ARRIVAL_DELAY')




In [7]:
from pyspark.sql import functions as fn
flights_df.groupBy('CANCELLED').agg(fn.count('*')).show()

+---------+--------+
CANCELLED|count(1)|
+---------+--------+
 0| 5729195|
+---------+--------+

In [8]:
flights_df.select("DISTANCE").rdd.max()[0]

Out[20]: 4983

In [9]:
flights_df.select("DISTANCE").rdd.min()[0]

Out[21]: 31

In [10]:
from pyspark.sql import functions as fn
flights_df.groupBy('Airline_Numeric').agg(fn.count('*')).show()

+---------------+--------+
Airline_Numeric|count(1)|
+---------------+--------+
 8.0| 194648|
 0.0| 1245812|
 7.0| 262772|
 1.0| 872057|
 4.0| 556746|
 11.0| 90248|
 3.0| 578393|
 2.0| 715065|
 10.0| 115375|
 13.0| 61369|
 6.0| 279607|
 5.0| 509150|
 9.0| 171852|
 12.0| 76101|
+---------------+--------+

In [11]:
flights_df.select('Airline_OHE').take(5)


Out[23]: [Row(Airline_OHE=SparseVector(13, {9: 1.0})),
 Row(Airline_OHE=SparseVector(13, {2: 1.0})),
 Row(Airline_OHE=SparseVector(13, {8: 1.0})),
 Row(Airline_OHE=SparseVector(13, {2: 1.0})),
 Row(Airline_OHE=SparseVector(13, {9: 1.0}))]

In [12]:
flights_df.limit(10).toPandas()

In [13]:
flights_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- Flight_Delayed: integer (nullable = false)
-- Distance_Bucket: double (nullable = true)
-- Flight_Distance: string (nullable = true)
-- Airline_Numeric: double (nullable = false)
-- Airline_OHE: vector (nullable = true)
-- OA_Numeric: double (nullable = false)
-- Origin_Airport_OHE: vector (nullable = true)
-- DA_Numeric: double (nullable = false)
-- Destination_Airport_OHE: vector (nullable = true)

In [14]:
training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)

In [15]:
major_df = training_df.filter(col("Flight_Delayed") == 0)
minor_df = training_df.filter(col("Flight_Delayed") == 1)

In [16]:
ratio = major_df.count()/minor_df.count()
print(ratio)

4.4276492398259215

In [17]:
sampled_majority_df = major_df.sample(False, 1.2/ratio, seed=5)
combined_df_2 = sampled_majority_df.unionAll(minor_df)
#combined_df_2.show()

In [18]:
testing_df.select([count(when(isnull(c), c)).alias(c) for c in training_df.columns]).show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|CANCELLED|Flight_Delayed|Distance_Bucket|Flight_Distance|Airline_Numeric|Airline_OHE|OA_Numeric|Origin_Airport_OHE|DA_Numeric|Destination_Airport_OHE|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+-----------+----------+------------------+----------+-----------------------+

In [19]:
combined_df_2.groupBy('Flight_Delayed').agg(fn.count('*')).show()

+--------------+--------+
Flight_Delayed|count(1)|
+--------------+--------+
 1| 949914|
 0| 1139995|
+--------------+--------+

In [20]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
columns = ["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED","Flight_Delayed"]
corrDf = flights_df.select("MONTH", "DAY", "DAY_OF_WEEK", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED","Flight_Delayed")
from pyspark.mllib.stat import Statistics
import pandas as pd

df_rdd = corrDf.rdd.map(lambda row: row[0:])
corr_mat = Statistics.corr(df_rdd, method='pearson')
corr_mat_df = pd.DataFrame(corr_mat,columns=corrDf.columns, index=corrDf.columns)
corr_mat_df
# vector_col = "corr_features"
# assembler = VectorAssembler(inputCols=corrDf.columns, outputCol=vector_col)
# df_vector = assembler.transform(corrDf).select(vector_col)
# matrix = Correlation.corr(df_vector, vector_col,'pearson')
# result = matrix.collect()[0]["pearson({})".format(vector_col)].values


,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,Distance_Bucket,SCHEDULED_ARRIVAL,CANCELLED,Flight_Delayed
MONTH,1.000000,0.005573,-0.009426,-0.000233,0.004872,-0.010354,NaN,-0.036677
DAY,0.005573,1.000000,0.002182,-0.001780,0.002723,-0.003868,NaN,-0.006203
DAY_OF_WEEK,-0.009426,0.002182,1.000000,0.007754,0.013961,0.006887,NaN,-0.009072
SCHEDULED_DEPARTURE,-0.000233,-0.001780,0.007754,1.000000,-0.024674,0.705120,NaN,0.166976
Distance_Bucket,0.004872,0.002723,0.013961,-0.024674,1.000000,0.027382,NaN,0.021078
SCHEDULED_ARRIVAL,-0.010354,-0.003868,0.006887,0.705120,0.027382,1.000000,NaN,0.140894
CANCELLED,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
Flight_Delayed,-0.036677,-0.006203,-0.009072,0.166976,0.021078,0.140894,NaN,1.000000


In [21]:
result

Out[45]: array([ 1. , 0.0055731 , -0.0094256 , ..., 0.14089393,
 nan, 1. ])

In [22]:
dim = len(result)
cor_mat = result.reshape((dim,dim )) 

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-1573592700932255> in <module> 
 1 dim = len ( result ) 
 ----> 2 cor_mat = result . reshape ( ( dim , dim ) ) 

 ValueError : cannot reshape array of size 1628176 into shape (1628176,1628176)

In [23]:
df = pd.DataFrame(result.reshape(-1, len(corrDf.columns)), columns=corrDf.columns, index=corrDf.columns)
df

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in create_block_manager_from_blocks (blocks, axes) 
 1652 
 -> 1653 mgr = BlockManager ( blocks , axes ) 
 1654 mgr . _consolidate_inplace ( ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in __init__ (self, blocks, axes, do_integrity_check) 
 113 if do_integrity_check : 
 --> 114 self . _verify_integrity ( ) 
 115 

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in _verify_integrity (self) 
 310 if block . _verify_integrity and block . shape [ 1 : ] != mgr_shape [ 1 : ] : 
 --> 311 construction_error ( tot_items , block . shape [ 1 : ] , self . axes ) 
 312 if len ( self . items ) != tot_items : 

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in construction_error (tot_items, block_shape, axes, e) 
 1690 raise ValueError("Shape of passed values is {0}, indices imply {1}".format(
 -> 1691 passed, implied))
 1692 

 ValueError : Shape of passed values is (148016, 11), indices imply (11, 11)

During handling of the above exception, another exception occurred:

 ValueError Traceback (most recent call last)
 <command-1573592700932254> in <module> 
 ----> 1 df = pd . DataFrame ( result . reshape ( - 1 , len ( corrDf . columns ) ) , columns = corrDf . columns , index = corrDf . columns ) 
 2 df

 /databricks/python/lib/python3.7/site-packages/pandas/core/frame.py in __init__ (self, data, index, columns, dtype, copy) 
 422 else : 
 423 mgr = init_ndarray(data, index, columns, dtype=dtype,
 --> 424 copy=copy)
 425 
 426 # For data is list-like, or Iterable (will consume into list) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/construction.py in init_ndarray (values, index, columns, dtype, copy) 
 165 values = maybe_infer_to_datetimelike ( values ) 
 166 
 --> 167 return create_block_manager_from_blocks ( [ values ] , [ columns , index ] ) 
 168 
 169 

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in create_block_manager_from_blocks (blocks, axes) 
 1658 blocks = [ getattr ( b , 'values' , b ) for b in blocks ] 
 1659 tot_items = sum ( b . shape [ 0 ] for b in blocks ) 
 -> 1660 construction_error ( tot_items , blocks [ 0 ] . shape [ 1 : ] , axes , e ) 
 1661 
 1662 

 /databricks/python/lib/python3.7/site-packages/pandas/core/internals/managers.py in construction_error (tot_items, block_shape, axes, e) 
 1689 raise ValueError ( "Empty data passed with indices specified." ) 
 1690 raise ValueError("Shape of passed values is {0}, indices imply {1}".format(
 -> 1691 passed, implied))
 1692 
 1693 

 ValueError : Shape of passed values is (148016, 11), indices imply (11, 11)

In [24]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="features")

In [25]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidatorModel
bce = BinaryClassificationEvaluator(labelCol='Flight_Delayed', metricName='areaUnderROC')
from pyspark.ml import evaluation
lr_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
rf_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")

In [26]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.75, thresholds=[0.25, 0.75])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)
#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))


Out[12]: 0.7331472575743154

In [27]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[13]: 0.8158581780448033

In [28]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.82 1.00 0.90 467809
 1 0.50 0.00 0.00 105586

 micro avg 0.82 0.82 0.82 573395
 macro avg 0.66 0.50 0.45 573395
weighted avg 0.76 0.82 0.73 573395

Out[14]: array([[467802, 7],
 [105579, 7]])

In [29]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.9, thresholds=[0.1, 0.9])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)
#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[15]: 0.7331239572897557

In [30]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[16]: 0.8158581780448033

In [31]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
 precision recall f1-score support

 0 0.82 1.00 0.90 467809
 1 0.00 0.00 0.00 105586

 micro avg 0.82 0.82 0.82 573395
 macro avg 0.41 0.50 0.45 573395
weighted avg 0.67 0.82 0.73 573395

Out[17]: array([[467809, 0],
 [105586, 0]])

In [32]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.6, thresholds=[0.4, 0.6])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)
#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[18]: 0.7562877665074327

In [33]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[19]: 0.7930553981112497

In [34]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.83 0.94 0.88 467809
 1 0.35 0.14 0.20 105586

 micro avg 0.79 0.79 0.79 573395
 macro avg 0.59 0.54 0.54 573395
weighted avg 0.74 0.79 0.76 573395

Out[20]: array([[439587, 28222],
 [ 90439, 15147]])

In [35]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.5, thresholds=[0.5, 0.5])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)
#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[21]: 0.7131270751632838

In [36]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[22]: 0.6868772835479904

In [37]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.86 0.73 0.79 467809
 1 0.29 0.48 0.36 105586

 micro avg 0.69 0.69 0.69 573395
 macro avg 0.58 0.61 0.58 573395
weighted avg 0.76 0.69 0.71 573395

Out[23]: array([[343187, 124622],
 [ 54921, 50665]])

In [38]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.4, thresholds=[0.6, 0.4])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)
#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[24]: 0.535160971769744

In [39]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[25]: 0.4896380331185308

In [40]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.90 0.42 0.57 467809
 1 0.24 0.79 0.36 105586

 micro avg 0.49 0.49 0.49 573395
 macro avg 0.57 0.61 0.47 573395
weighted avg 0.78 0.49 0.54 573395

Out[26]: array([[196959, 270850],
 [ 21789, 83797]])

In [41]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.25, thresholds=[0.75, 0.25])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)
#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[27]: 0.12621808813216837

In [42]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[28]: 0.21746265663286216

In [43]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.94 0.04 0.08 467809
 1 0.19 0.99 0.32 105586

 micro avg 0.22 0.22 0.22 573395
 macro avg 0.57 0.52 0.20 573395
weighted avg 0.80 0.22 0.13 573395

Out[29]: array([[ 20318, 447491],
 [ 1212, 104374]])

In [44]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.1, thresholds=[0.9, 0.1])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)

#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[30]: 0.057324106298078326

In [45]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[31]: 0.1841679819321759

In [46]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 1.00 0.00 0.00 467809
 1 0.18 1.00 0.31 105586

 micro avg 0.18 0.18 0.18 573395
 macro avg 0.59 0.50 0.16 573395
weighted avg 0.85 0.18 0.06 573395

Out[32]: array([[ 15, 467794],
 [ 0, 105586]])

In [47]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.35, thresholds=[0.65, 0.35])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)

#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[11]: 0.3927928949226273

In [48]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[12]: 0.37591363719599924

In [49]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.92 0.26 0.40 467809
 1 0.21 0.90 0.35 105586

 micro avg 0.38 0.38 0.38 573395
 macro avg 0.57 0.58 0.37 573395
weighted avg 0.79 0.38 0.39 573395

Out[13]: array([[120986, 346823],
 [ 11025, 94561]])

In [50]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.45, thresholds=[0.55, 0.45])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)

#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_df))

Out[20]: 0.642683920645353

In [51]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[21]: 0.5974363222560364

In [52]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.88 0.58 0.70 467809
 1 0.26 0.65 0.37 105586

 micro avg 0.60 0.60 0.60 573395
 macro avg 0.57 0.62 0.54 573395
weighted avg 0.77 0.60 0.64 573395

Out[22]: array([[273664, 194145],
 [ 36683, 68903]])

In [53]:
lr = LogisticRegression(featuresCol='features', labelCol='Flight_Delayed', regParam=0.1, threshold=0.55, thresholds=[0.45, 0.55])
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(combined_df_2)
lr_transform = lr_model.transform(testing_df)

#bce.evaluate(lr_pipeline.transform(testing_df))
lr_evaluator_f1.evaluate(lr_model.transform(testing_dfing_df))

Out[23]: 0.7499452531079315

In [54]:
lr_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
lr_evaluator_recall.evaluate(lr_model.transform(testing_df))

Out[24]: 0.7531178332563067

In [55]:
predictions_train = lr_model.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.84 0.86 0.85 467809
 1 0.32 0.30 0.31 105586

 micro avg 0.75 0.75 0.75 573395
 macro avg 0.58 0.58 0.58 573395
weighted avg 0.75 0.75 0.75 573395

Out[25]: array([[400384, 67425],
 [ 74136, 31450]])

In [56]:
lr_evaluator_precision = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
lr_evaluator_precision.evaluate(lr_model.transform(testing_df))

In [57]:
lr_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
lr_evaluator_accuracy.evaluate(lr_model.transform(testing_df))

In [59]:
rf = RandomForestClassifier(featuresCol='features', labelCol='Flight_Delayed', numTrees=3, maxDepth=4, impurity="gini")
rf_pipeline = Pipeline(stages=[va, rf])
rf_model = rf_pipeline.fit(training_df)
#bce.evaluate(rf_pipeline.transform(testing_df))
rf_transform = rf_model.transform(testing_df)
rf_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="f1")
rf_evaluator_f1.evaluate(rf_transform)

In [60]:
rf_evaluator_recall = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedRecall")
rf_evaluator_recall.evaluate(rf_pipeline.transform(testing_df))

In [61]:
rf_evaluator_Precision = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="weightedPrecision")
rf_evaluator_Precision.evaluate(rf_pipeline.transform(testing_df))

In [62]:
rf_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
rf_evaluator_accuracy.evaluate(rf_pipeline.transform(testing_df))

In [63]:
gbt = GBTClassifier(featuresCol='features', labelCol='Flight_Delayed')
gbt_pipeline = Pipeline(stages=[va, gbt]).fit(training_df)
#bce.evaluate(gbt_pipeline.transform(testing_df))
evaluator.evaluate(gbt_pipeline.transform(testing_df))

In [64]:
rf_pipe = Pipeline(stages=[rf])

In [65]:
paramGrid = ParamGridBuilder()\
  .addGrid(rf.maxBins, [25, 28, 31])\
  .addGrid(rf.maxDepth, [4, 6, 8])\
  .addGrid(rf.numTrees, [10, 20, 30])\
  .addGrid(rf.impurity, ["entropy", "gini"])\
  .build()

In [66]:
cv = CrossValidator()\
  .setEstimator(rf_pipe)\
  .setEvaluator(evaluator)\
  .setEstimatorParamMaps(paramGrid)\
  .setNumFolds(3)

In [67]:
cv_pipeline = Pipeline(stages=[va,cv]).fit(training_df)

In [68]:
evaluator.evaluate(cv_pipeline.transform(testing_df))

In [69]:
from pyspark.ml import classification

mlp = classification.MultilayerPerceptronClassifier(featuresCol='features', labelCol='Flight_Delayed', layers=[4,2])
mlp_pipeline = Pipeline(stages=[va, mlp]).fit(training_df)
evaluator.evaluate(mlp_pipeline.transform(testing_df))

In [70]:
from pyspark.ml.classification import LinearSVC

svm = LinearSVC(featuresCol='features', labelCol='Flight_Delayed', maxIter=10, regParam=0.1)
svm_pipeline = Pipeline(stages=[va, svm]).fit(training_df)
evaluator.evaluate(svm_pipeline.transform(testing_df))


In [71]:
from pyspark.ml import feature

pca = PCA(k=2, inputCol="features", outputCol="pca_features")
 model = pca.fit(df)
 model.transform(df).collect()

In [72]:
flights_df_sample = flights_df.sample(True, 0.5, 42)

In [73]:
shape = ((flights_df_sample.count(), len(flights_df_sample.columns)))
print('The shape of flights_df_sample:', shape)

In [74]:
flights_df_sample.printSchema()

In [75]:
flights_df_sample_pandas = flights_df_sample.toPandas()

In [76]:
flights_df_sample.groupBy('Flight_Delayed').agg(fn.count('*')).show()

In [77]:
flights_df.groupBy('Flight_Delayed').agg(fn.count('*')).show()

In [78]:
from pyspark.ml.stat import Correlation

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED", "Flight_Delayed"], outputCol=vector_col)
df_vector = assembler.transform(flights_df).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)


In [79]:
matrix.show()

In [80]:
sns.distplot(flights_df_sample_pandas['MONTH'])
display()

In [81]:
sns.distplot(flights_df_sample_pandas['DAY'])
display()

In [82]:
sns.distplot(flights_df_sample_pandas['DAY_OF_WEEK'])
display()

In [83]:
flights_df_sample_pandas['AIRLINE'].value_counts().plot(kind='bar')
display()

In [84]:
airline_count  = flights_df_sample_pandas['AIRLINE'].value_counts()
airline_count = airline_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(airline_count.index, airline_count.values, alpha=0.8)
display()

In [85]:
oa_count  = flights_df_sample_pandas['ORIGIN_AIRPORT'].value_counts()
oa_count = oa_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(oa_count.index, oa_count.values, alpha=0.8)
display()

In [86]:
da_count  = flights_df_sample_pandas['DESTINATION_AIRPORT'].value_counts()
da_count = da_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(da_count.index, da_count.values, alpha=0.8)
display()

In [87]:
sns.distplot(flights_df_sample_pandas['SCHEDULED_DEPARTURE'])
display()

In [88]:
sns.distplot(flights_df_sample_pandas['DEPARTURE_DELAY'])
display()

In [89]:
sns.distplot(flights_df_sample_pandas['DISTANCE'])
display()

In [90]:
sns.distplot(flights_df_sample_pandas['Distance_Bucket'])
display()

In [91]:
sns.distplot(flights_df_sample_pandas['SCHEDULED_ARRIVAL'])
display()

In [92]:
sns.distplot(flights_df_sample_pandas['CANCELLED'])
display()

In [93]:
sns.distplot(flights_df_sample_pandas['Flight_Delayed'])
display()

In [94]:

sns.barplot(x="MONTH", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [95]:
df3 = flights_df.groupBy('MONTH').avg('DEPARTURE_DELAY').toPandas()
df3
fig4 = sns.barplot(x = 'MONTH', y = 'avg(DEPARTURE_DELAY)', data = df3)
display()

In [96]:
df3.head()

In [97]:
sns.barplot(x="MONTH", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

In [98]:
sns.barplot(x="DAY", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [99]:
sns.barplot(x="DAY", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

In [100]:
sns.barplot(x="DAY_OF_WEEK", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [101]:
sns.barplot(x="DAY_OF_WEEK", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()

In [102]:
sns.barplot(x="AIRLINE", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [103]:
sns.barplot(x="ORIGIN_AIRPORT", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [104]:
sns.barplot(x="SCHEDULED_DEPARTURE", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [105]:
sns.barplot(x="Distance_Bucket", y="DEPARTURE_DELAY", data=flights_df_sample_pandas)
display()

In [106]:
sns.barplot(x="AIRLINE", y="DEPARTURE_DELAY", hue="Flight_Distance", data=flights_df_sample_pandas)
display()